<div style='float: right'><img src='pic/hotaru.png'/></div>
## <div id='hotaru' />ホタルビーム

In [ ]:
from pulp import *
from unionfind import unionfind
from ortoolpy import addvar, addvars, addbinvar, addbinvars
hint = [s.split(',') for s in """\
0,0,0,R
0,3,2,B
0,5,3,L
2,1,?,B
2,5,4,L
3,3,1,T
4,0,1,T
4,3,0,L
5,4,?,T""".split('\n')]
nw, nh = 6, 6
L = 9 # limit length from '?'
rh, rw, rh1, rw1 = range(nh), range(nw), range(nh-1), range(nw-1)

### 問題
* 全ての白丸の黒点から点線上に線をのばして白丸の黒点でないところにつなげます
* どの線も白丸以外のところで、途切れたり交差したり枝分かれしたりしてはいけません
* 線で全体がひとつながりにならなければいけません
* 数字は、その白丸の黒点から出る線が白丸につながるまでに曲がる回数を表します

### 変数
* 省略

### 制約
* 省略

In [ ]:
bh = [[(-1, 0) for j in rw] for i in rh] # (id, dir) of hint
for k, h in enumerate(hint):
    hint[k] = hc = [int(h[0]), int(h[1]), -2 if h[2] == '?' else int(h[2]), 'LTRB'.index(h[3])]
    bh[hc[0]][hc[1]] = (k, hc[3])
m = LpProblem()
cc = [[[LpAffineExpression() for j in rw] for i in rh], # check node overlap \
      [[LpAffineExpression() for j in rw1] for i in rh], # check h_line overlap \
      [[LpAffineExpression() for j in rw] for i in rh1]] # check v_line overlap
vas = []
def make(cands, y, x, n, d, p0, q0):
    dy, dx = [0, -1, 0, 1][d], [-1, 0, 1, 0][d]
    ny, nx = y + dy, x + dx
    if n == -1 or (ny, nx) in p0 or not (0 <= ny < nh and 0 <= nx < nw) or len(p0) > L:
        return
    p, q = p0 + [(ny, nx)], q0 + [(0, ny, nx), (1, ny, nx), (0, y, x), (1, y, x)][d:d + 1]
    if bh[ny][nx][0] >= 0:
        if n <= 0 and d != (bh[ny][nx][1] + 2) % 4:
            cands.append((p, q))
        return
    for k in range(4):
        make(cands, ny, nx, n if d == k else n - 1, k, p, q)
for h in hint:
    cands = []
    make(cands, h[0], h[1], h[2], h[3], [(h[0], h[1])], [])
    vv = addbinvars(len(cands))
    m += lpSum(vv) == 1
    for i in range(len(cands)):
        vas.append((vv[i], cands[i]))
        for j, (w,y,x) in enumerate(cands[i][1]):
                cc[0][cands[i][0][j][0]][cands[i][0][j][1]] += vv[i]
                cc[w+1][y][x] += vv[i]
for i in rh:
    for j in rw:
        m += cc[0][i][j] <= 1
        if j < nw - 1:
            m += cc[1][i][j] <= 1
        if i < nh - 1:
            m += cc[2][i][j] <= 1
while True:
    %time m.solve()
    u = unionfind(len(hint))
    l = []
    for va, cand in vas:
        if value(va) > 0.5:
            l.append(va)
            (y1, x1), (y2, x2) = cand[0][0], cand[0][-1]
            u.unite(bh[y1][x1][0], bh[y2][x2][0])
    if all([u.issame(0, k) for k in range(1,len(hint))]):
        break
    m += lpSum(l) <= len(l) - 1
bd = [[' '] * (2 * nw - 1) for i in range(2 * nh - 1)]
for h in hint:
    bd[h[0] * 2][h[1] * 2] = '?' if h[2] < 0 else str(h[2])
for va, cand in vas:
    if value(va) > 0.5:
        for w,y,x in cand[1]:
            bd[2 * y + w][2 * x + 1 - w] = '-|'[w]
for b in bd:
    print(' '.join(b))